In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good")
else:
    print("There might be a problem with the API key")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good


In [3]:
links = fetch_website_links('https://huggingface.co')
links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2.5',
 '/nvidia/personaplex-7b-v1',
 '/Tongyi-MAI/Z-Image',
 '/Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice',
 '/deepseek-ai/DeepSeek-OCR-2',
 '/models',
 '/spaces/Qwen/Qwen3-TTS',
 '/spaces/multimodalart/qwen-image-multiple-angles-3d-camera',
 '/spaces/mrfakename/Z-Image-Turbo',
 '/spaces/black-forest-labs/FLUX.2-klein-9B',
 '/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast',
 '/spaces',
 '/datasets/opendatalab/ChartVerse-SFT-1800K',
 '/datasets/Qwen/DeepPlanning',
 '/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b',
 '/datasets/mercor/apex-agents',
 '/datasets/sojuL/RubricHub_v1',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon',


In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt="""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
"""

    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt('https://huggingface.co'))


Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/moonshotai/Kimi-K2.5
/nvidia/personaplex-7b-v1
/Tongyi-MAI/Z-Image
/Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
/deepseek-ai/DeepSeek-OCR-2
/models
/spaces/Qwen/Qwen3-TTS
/spaces/multimodalart/qwen-image-multiple-angles-3d-camera
/spaces/mrfakename/Z-Image-Turbo
/spaces/black-forest-labs/FLUX.2-klein-9B
/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast
/spaces
/datasets/opendatalab/ChartVerse-SFT-1800K
/datasets/Qwen/DeepPlanning
/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b
/datasets/mercor/apex-agents
/datasets/sojuL/RubricHub_v1
/datasets
/join
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/enterpris

In [7]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [8]:
select_relevant_links('https://huggingface.co')

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community page', 'url': 'https://huggingface.co/join'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'social media - Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'social media - LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'source code', 'url': 'https://github.com/huggingface'}]}

In [9]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [10]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
moonshotai/Kimi-K2.5
Updated
about 17 hours ago
•
25.4k
•
1.17k
nvidia/personaplex-7b-v1
Updated
2 days ago
•
54.5k
•
1.5k
Tongyi-MAI/Z-Image
Updated
2 days ago
•
1.92k
•
696
Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
Updated
1 day ago
•
181k
•
796
deepseek-ai/DeepSeek-OCR-2
Updated
about 22 hours ago
•
45.3k
•
554
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.03k
Qwen3-TTS Demo
🎙
1.03k
Transform text into natural-sounding speech with custom voices
Running
on
Zero
Featured
1.27k
Qwen Image Multiple Angles 3D Camera
🎥
1.27k
Adjust came

In [18]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short informational prompt about the company for llm related tasks, information and insights.
The prompt will be used to be fed into another llm, so that the other llm has better insights
and understanding of the company presented.
Respond in markdown without code blocks.
Include only valuable information for informational prompts.
"""

In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2.5\nUpdated\nabout 18 hours ago\n•\n25.4k\n•\n1.17k\nnvidia/personaplex-7b-v1\nUpdated\n2 days ago\n•\n54.5k\n•\n1.5k\nTongyi-MAI/Z-Image\nUpdated\n2 days ago\n•\n1.92k\n•\n696\nQwen/Qwen3-TTS-12Hz-1.7B-CustomVoice\nUpdated\n1 day ago\n•\n181k\n•\n797\ndeepseek-ai/DeepSeek-OCR-2\nUpdated\nabout 22 hours ago\n•\n45.3k\n•\n555\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nF

In [21]:
def create_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

In [ ]:
name_input = gr.Textbox(label='Company name:')
url_input = gr.Textbox(label='Landing page URL including http:// or https://')
message_output = gr.Markdown(label='Response')

view = gr.Interface(
    fn=create_brochure,
    title='AI Prompt Generator',
    inputs=[name_input, url_input],
    outputs=[message_output],
    examples=[
            ["Marigo App", "https://marigoapp.com"],
        ], 
    flagging_mode="never"
)

view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://78ddf5962565560380.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Selecting relevant links for https://marigoapp.com by calling gpt-5-nano
Found 7 relevant links
Selecting relevant links for https://marigoapp.com by calling gpt-5-nano
Found 4 relevant links
Selecting relevant links for https://wolt.com by calling gpt-5-nano
Found 7 relevant links
